In [1]:
! du -h ../data/processed/pagecount

4.6G	../data/processed/pagecount


In [2]:
pagecount = spark.read.parquet("../data/processed/pagecount")

In [3]:
pagecount.show(n=5)

+-------+-------------------+-----+
|page_id|         visit_time|count|
+-------+-------------------+-----+
|   1436|2019-07-02 00:00:00|   41|
|   1436|2019-07-02 01:00:00|   36|
|   1436|2019-07-02 02:00:00|   40|
|   1436|2019-07-02 03:00:00|   31|
|   1436|2019-07-02 04:00:00|   25|
+-------+-------------------+-----+
only showing top 5 rows



In [4]:
import pyspark.sql.functions as F
daily = (
    pagecount
    .select("page_id", F.date_format("visit_time", "yyyy-MM-dd").alias("date"), "count")
    .groupBy("page_id", "date")
    .agg(F.sum("count").alias("count"))
)
daily.show()

+-------+----------+-----+
|page_id|      date|count|
+-------+----------+-----+
|   1436|2018-06-27| 1154|
|   1436|2018-11-08| 1598|
|   1436|2018-03-07| 1617|
|   2085|2019-05-12|  726|
|   2085|2018-11-28| 1176|
|   2085|2018-02-15| 1331|
|   2085|2019-01-11| 1040|
|   2085|2019-03-18|  952|
|   2085|2018-04-16| 1414|
|   2761|2018-12-31|  130|
|   2761|2018-05-29|  338|
|   2761|2019-02-17|  215|
|   2822|2019-01-17|  166|
|   2822|2019-04-16|  133|
|   2844|2019-08-28|  343|
|   2844|2018-12-14|  251|
|   2844|2019-02-13|  268|
|   3138|2018-09-07| 2096|
|   3138|2019-03-03| 1484|
|   3457|2018-05-30| 2945|
+-------+----------+-----+
only showing top 20 rows



In [17]:
daily.repartitionByRange(2, "page_id", "date").write.parquet(
    "../data/enwiki/pagecount_daily_v2", mode="overwrite"
)

In [18]:
! tree -h ../data/enwiki/pagecount_daily_v2/

../data/enwiki/pagecount_daily_v2/
├── [   0]  _SUCCESS
├── [235M]  part-00000-97cd53e2-dab7-4dc1-bb1c-32d45d1e7669-c000.snappy.parquet
└── [249M]  part-00001-97cd53e2-dab7-4dc1-bb1c-32d45d1e7669-c000.snappy.parquet

0 directories, 3 files
